# Pipeline de Catégorisation Multi-label de Dépôts GitHub (Version Simplifiée)

**Objectif :** Ce notebook utilise une approche ciblée en se basant **uniquement sur la description et le langage** des dépôts pour :
1.  **Créer une base de 100 catégories** en analysant un large ensemble de dépôts GitHub.
2.  **Classifier chaque dépôt** en lui assignant de 1 à N catégories pertinentes, en se basant sur un score de similarité sémantique.

In [1]:
!pip install requests pandas ijson tqdm transformers torch scikit-learn numpy matplotlib seaborn umap-learn

In [2]:
# !pip install pandas tqdm transformers torch scikit-learn numpy matplotlib seaborn umap-learn

import pandas as pd
import json
import numpy as np
from tqdm.notebook import tqdm

# Machine Learning & Embeddings
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# --- CONFIGURATION ---

# MODIFIEZ ICI LE NOM DE VOTRE FICHIER CSV
INPUT_FILE_CSV = "github_repos_categorized.csv" # <--- METTEZ VOTRE NOM DE FICHIER ICI

# Fichiers de sortie
CATEGORIES_FILE = "github_categories_database.json"
OUTPUT_FILE = "github_repos_multilabel_categorized_from_csv.csv"

# Paramètres du script
N_CATEGORIES = 100
TOP_N_CATEGORIES = 10
SIMILARITY_THRESHOLD = 0.5

# Device pour PyTorch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilisation du device : {device}")

Utilisation du device : cpu


In [3]:
# --- CHARGEMENT DIRECT DEPUIS LE FICHIER CSV ---
try:
    all_repos_df = pd.read_csv(INPUT_FILE_CSV)
    print(f"Fichier '{INPUT_FILE_CSV}' chargé avec succès.")
except FileNotFoundError:
    print(f"ERREUR : Le fichier '{INPUT_FILE_CSV}' n'a pas été trouvé. Vérifiez le nom et l'emplacement du fichier.")
    # On crée un DataFrame vide pour éviter que les cellules suivantes ne plantent
    all_repos_df = pd.DataFrame() 

if not all_repos_df.empty:
    # On s'assure que les colonnes nécessaires sont présentes
    required_columns = ['description', 'language']
    if not all(col in all_repos_df.columns for col in required_columns):
        print(f"ERREUR : Le CSV doit contenir les colonnes suivantes : {required_columns}")
    else:
        # On supprime les dépôts sans description, car ils sont impossibles à classifier
        all_repos_df.dropna(subset=['description'], inplace=True)
        all_repos_df.reset_index(drop=True, inplace=True)
        print(f"{len(all_repos_df)} dépôts avec description prêts à être traités.")
        print("Aperçu des données :")
        print(all_repos_df.head())

Fichier 'github_repos_categorized.csv' chargé avec succès.
1920 dépôts avec description prêts à être traités.
Aperçu des données :
                              full_name  \
0  mdbootstrap/bootstrap-toggle-buttons   
1                      cloudfuji/kandan   
2                          doug/depthjs   
3            codrops/ModalWindowEffects   
4                adactio/Pattern-Primer   

                                            html_url  \
0  https://github.com/mdbootstrap/bootstrap-toggl...   
1                https://github.com/cloudfuji/kandan   
2                    https://github.com/doug/depthjs   
3      https://github.com/codrops/ModalWindowEffects   
4          https://github.com/adactio/Pattern-Primer   

                                         description    language  stars  \
0  Bootstrap-toggle-buttons has moved to https://...  JavaScript   1013   
1                       A Cloudfuji chat application  JavaScript   1005   
2  DepthJS allows any web page to interact with 

In [4]:
if not all_repos_df.empty:
    def build_input_text(row):
        """Construit le texte d'entrée pour le modèle."""
        lang = str(row['language']) if pd.notna(row['language']) else ''
        desc = str(row['description']) if pd.notna(row['description']) else ''
        return f"Langage : {lang}. Description : {desc}"

    tqdm.pandas(desc="Construction du texte d'entrée")
    all_repos_df['full_text'] = all_repos_df.progress_apply(build_input_text, axis=1)

    # --- Fonctions d'Embedding ---
    def mean_pooling(model_output, attention_mask):
        token_embeddings = model_output[0]
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    print("\nChargement du modèle 'all-MiniLM-L6-v2'...")
    tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
    model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2').to(device)
    print("Modèle chargé.")

    def get_embeddings(sentences, batch_size=32):
        all_embeddings = []
        for i in tqdm(range(0, len(sentences), batch_size), desc="Génération des Embeddings"):
            batch = sentences[i:i+batch_size]
            encoded_input = tokenizer(batch, padding=True, truncation=True, return_tensors='pt').to(device)
            with torch.no_grad():
                model_output = model(**encoded_input)
            sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
            sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
            all_embeddings.append(sentence_embeddings.cpu().numpy())
        return np.vstack(all_embeddings)

    corpus = all_repos_df['full_text'].tolist()
    embeddings = get_embeddings(corpus)
    print(f"\nEmbeddings générés. Shape: {embeddings.shape}")

Construction du texte d'entrée:   0%|          | 0/1920 [00:00<?, ?it/s]


Chargement du modèle 'all-MiniLM-L6-v2'...


2025-10-12 16:34:58.435477: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Modèle chargé.


Génération des Embeddings:   0%|          | 0/60 [00:00<?, ?it/s]


Embeddings générés. Shape: (1920, 384)


In [5]:
if not all_repos_df.empty:
    print(f"--- PHASE A : Création de la base de {N_CATEGORIES} catégories ---")

    kmeans = KMeans(n_clusters=N_CATEGORIES, random_state=42, n_init=10)
    all_repos_df['cluster_id'] = kmeans.fit_predict(embeddings)
    print("Clustering terminé.")
    
    category_embeddings = kmeans.cluster_centers_

--- PHASE A : Création de la base de 100 catégories ---
Clustering terminé.


In [6]:
if not all_repos_df.empty:
    vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
    tfidf_matrix = vectorizer.fit_transform(all_repos_df['full_text'])
    terms = vectorizer.get_feature_names_out()

    categories_database = []
    for i in tqdm(range(N_CATEGORIES), desc="Nommage des catégories"):
        cluster_df = all_repos_df[all_repos_df['cluster_id'] == i]
        indices = cluster_df.index
        mean_tfidf = np.asarray(tfidf_matrix[indices].mean(axis=0)).ravel()
        top_indices = mean_tfidf.argsort()[-5:][::-1]
        keywords = [terms[j] for j in top_indices]
        
        if not cluster_df['language'].mode().empty:
            top_language = cluster_df['language'].mode()[0]
        else:
            top_language = "Tool"
        
        suggested_name = f"{top_language} - {', '.join(keywords)}"
        
        categories_database.append({
            "category_id": i,
            "category_name": suggested_name,
            "embedding": category_embeddings[i].tolist()
        })

    with open(CATEGORIES_FILE, 'w', encoding='utf-8') as f:
        json.dump(categories_database, f, ensure_ascii=False, indent=4)

    print(f"\n✅ Base de {N_CATEGORIES} catégories créée et sauvegardée dans '{CATEGORIES_FILE}'")

Nommage des catégories:   0%|          | 0/100 [00:00<?, ?it/s]


✅ Base de 100 catégories créée et sauvegardée dans 'github_categories_database.json'


In [7]:
if not all_repos_df.empty:
    print(f"--- PHASE B : Classification Multi-label ---")

    with open(CATEGORIES_FILE, 'r', encoding='utf-8') as f:
        categories_db = json.load(f)

    category_names = [cat['category_name'] for cat in categories_db]
    category_embeddings = np.array([cat['embedding'] for cat in categories_db])
    print(f"Base de {len(category_names)} catégories chargée.")

    print("Calcul des scores de similarité...")
    similarity_matrix = cosine_similarity(embeddings, category_embeddings)
    print(f"Matrice de similarité calculée. Shape: {similarity_matrix.shape}")

--- PHASE B : Classification Multi-label ---
Base de 100 catégories chargée.
Calcul des scores de similarité...
Matrice de similarité calculée. Shape: (1920, 100)


In [8]:
if not all_repos_df.empty:
    def assign_categories(similarity_scores, category_names, top_n, threshold):
        top_indices = np.argsort(similarity_scores)[-top_n:][::-1]
        assigned = []
        for index in top_indices:
            if similarity_scores[index] >= threshold:
                assigned.append({
                    "category": category_names[index],
                    "score": round(float(similarity_scores[index]), 3)
                })
        return assigned if assigned else [{"category": "Unclassified", "score": 0.0}]

    all_repos_df['assigned_categories'] = [
        assign_categories(scores, category_names, TOP_N_CATEGORIES, SIMILARITY_THRESHOLD)
        for scores in tqdm(similarity_matrix, desc="Classification des dépôts")
    ]

    print("\n✅ Classification multi-label terminée.")

Classification des dépôts:   0%|          | 0/1920 [00:00<?, ?it/s]


✅ Classification multi-label terminée.


In [9]:
if not all_repos_df.empty:
    # On enlève les colonnes de travail avant de sauvegarder
    columns_to_drop = ['full_text', 'cluster_id']
    all_repos_df_to_save = all_repos_df.drop(columns=[col for col in columns_to_drop if col in all_repos_df.columns])
    
    all_repos_df_to_save.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')
    print(f"Résultats finaux sauvegardés dans '{OUTPUT_FILE}'.")

    # Afficher les 15 premiers résultats de manière lisible
    for index, row in all_repos_df.head(15).iterrows():
        print(f"\n--- {row.get('full_name', 'Repo Sans Nom')} ---")
        print(f"Langage: {row.get('language', 'N/A')}")
        print(f"Description: {row.get('description', 'N/A')}")
        print("Catégories Assignées:")
        for cat in row.get('assigned_categories', []):
            print(f"  - {cat['category']} (Score: {cat['score']})")

Résultats finaux sauvegardés dans 'github_repos_multilabel_categorized_from_csv.csv'.

--- mdbootstrap/bootstrap-toggle-buttons ---
Langage: JavaScript
Description: Bootstrap-toggle-buttons has moved to https://github.com/nostalgiaz/bootstrap-switch
Catégories Assignées:
  - JavaScript - javascript, description, langage, js, file (Score: 0.506)

--- cloudfuji/kandan ---
Langage: JavaScript
Description: A Cloudfuji chat application
Catégories Assignées:
  - JavaScript - javascript, chat, description, langage, steam (Score: 0.855)
  - JavaScript - javascript, langage, description, web, node (Score: 0.746)
  - JavaScript - javascript, langage, description, css, web (Score: 0.724)
  - JavaScript - javascript, browser, image, langage, description (Score: 0.719)
  - JavaScript - javascript, editor, langage, description, plugin (Score: 0.687)
  - JavaScript - node, javascript, js, graphql, s3 (Score: 0.654)
  - JavaScript - javascript, description, langage, js, file (Score: 0.646)
  - C - lan